In [ ]:
import tarfile
import json
from bz2 import BZ2File
import random
import os
import re
import time
from openai import OpenAI
import csv


In [ ]:
fname = "cmv.tar.bz2"
# make sure you have downloaded the dataset from https://chenhaot.com/data/cmv/cmv.tar.bz2

if not os.path.isfile(fname):
    raise FileNotFoundError(f"Dataset file {fname} not found. Please check your directory.")

tar = tarfile.open(fname, mode="r:bz2")


# extract the train and test pairs
train_fname = "pair_task/train_pair_data.jsonlist.bz2"
test_fname = "pair_task/heldout_pair_data.jsonlist.bz2"

train_bzlist = tar.extractfile(train_fname)
test_bzlist = tar.extractfile(test_fname)

# load JSON data
train_argument_pairs = [json.loads(line.decode('utf-8')) for line in BZ2File(train_bzlist)]
test_argument_pairs = [json.loads(line.decode('utf-8')) for line in BZ2File(test_bzlist)]



In [ ]:
def extract_argument_text(argument):
    """Extracts clean argument text from JSON structure."""
    if isinstance(argument, dict) and "comments" in argument:
        # extract first comment's body text
        if isinstance(argument["comments"], list) and len(argument["comments"]) > 0:
            return argument["comments"][0].get("body", "").strip()
    return ""

def format_prompt(sample):
    """Formats a single argument pair into a GPT-compatible prompt."""

    # extract components
    original_post = sample["op_text"]
    counterargument_1 = extract_argument_text(sample["positive"])  # Persuasive argument
    counterargument_2 = extract_argument_text(sample["negative"])  # Non-persuasive argument

    if not counterargument_1 or not counterargument_2:
        return None

    # shuffle order
    if random.random() > 0.5:
        counterargument_1, counterargument_2 = counterargument_2, counterargument_1
        correct_answer = "2"
    else:
        correct_answer = "1"

    prompt = f"""
You are an AI assistant analyzing persuasion in online discussions. Below is an original post (OP) and two counterarguments.

Your task:
1. Predict which argument is more persuasive, (1) or (2), taking into account the original post.
2. Justify your prediction based on reasoning, tone, and style.

---

Original Post (OP):
"{original_post}"

Counterargument 1:
"{counterargument_1}"

Counterargument 2:
"{counterargument_2}"

---

**Response Format (follow this exactly!):**
Prediction: [1 or 2]
Explanation: [Your reasoning]

"""

    return {"prompt": prompt, "correct_answer": correct_answer}

# === Test on a Sample ===
#formatted_sample = format_prompt(train_argument_pairs[0])

# Ensure the function worked correctly
'''if formatted_sample:
    print("Formatted GPT Prompt:")
    print(formatted_sample["prompt"])
    print(f"Correct Answer:{formatted_sample['correct_answer']}")
else:
    print("\n Skipped a sample due to missing argument text.")'''


In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def call_gpt(prompt, model="gpt-4o", max_retries=3):
    """
    Sends a prompt to the GPT (default value is 4o) and extracts:
    - Prediction (1 or 2)
    - Explanation for the prediction
    """
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {
                        "role": "system",
                        "content": (
                            "You are an AI assistant that analyzes persuasive arguments. "
                            "Given an original post and two counterarguments, you must: "
                            "1. Predict which argument is more persuasive (1 or 2), considering the original post. "
                            "2. Justify your prediction based on reasoning, tone, and style. "
                            "Your response must strictly follow this format:\n"
                            "Prediction: [1 or 2]\n"
                            "Explanation: [Your reasoning]"
                        )
                    },
                    {"role": "user", "content": prompt}
                ],
                temperature=0.7,
                max_tokens=250
            )

            # get the raw response text
            gpt_output = response.choices[0].message.content.strip()

            # extract the prediction
            match = re.search(r"Prediction:\s*(1|2)", gpt_output)
            prediction = match.group(1) if match else "Unknown"

            # extract explanation
            explanation = gpt_output.split("Explanation:", 1)[-1].strip() if "Explanation:" in gpt_output else gpt_output

            return {
                "prediction": prediction,
                "explanation": explanation,
                "raw_response": gpt_output
            }

        except Exception as e:
            print(f"Call failed (Attempt {attempt+1}/{max_retries}): {e}")
            time.sleep(2)

    return {
        "prediction": "Error",
        "explanation": "Failed to retrieve response",
        "raw_response": ""
    }



In [ ]:

num_samples = 500

# select a random sample from the data
sampled_data = random.sample(train_argument_pairs, num_samples)
gpt_prompts = [format_prompt(sample) for sample in sampled_data]


gpt_prompts = [p for p in gpt_prompts if p is not None]

output_file = "gpt_results_4o.csv"
results = []

for i, sample in enumerate(gpt_prompts):
    print(f"Processing sample {i+1}/{len(gpt_prompts)}...")


    response = call_gpt(sample["prompt"], model="gpt-4o")

    results.append({
        "original_post": sampled_data[i]["op_text"],
        "counterargument_1": sampled_data[i]["positive"],
        "counterargument_2": sampled_data[i]["negative"],
        "gpt_prediction": response["prediction"],
        "correct_answer": sample["correct_answer"],
        "explanation": response["explanation"]
    })

    time.sleep(1)

# save results
with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["original_post", "counterargument_1", "counterargument_2", "gpt_prediction", "correct_answer", "explanation"])
    writer.writeheader()
    writer.writerows(results)

print("Finished processing! Results saved.")

In [ ]:
# Same process but for 3.5
num_samples = 500

gpt_prompts = [format_prompt(sample) for sample in sampled_data]


gpt_prompts = [p for p in gpt_prompts if p is not None]

output_file = "old/gpt_results_3_5.csv"
results = []

for i, sample in enumerate(gpt_prompts):
    print(f"Processing sample {i+1}/{len(gpt_prompts)}...")


    response = call_gpt(sample["prompt"], model="gpt-3.5-turbo")

    results.append({
        "original_post": sampled_data[i]["op_text"],
        "counterargument_1": sampled_data[i]["positive"],
        "counterargument_2": sampled_data[i]["negative"],
        "gpt_prediction": response["prediction"],
        "correct_answer": sample["correct_answer"],
        "explanation": response["explanation"]
    })

    time.sleep(1)

# save results
with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["original_post", "counterargument_1", "counterargument_2", "gpt_prediction", "correct_answer", "explanation"])
    writer.writeheader()
    writer.writerows(results)

print("Finished processing! Results saved.")